In [1]:
#SAVE LOCATIION: CHANGE BEFORE EACH TEST
FOLDER = 'Data3'

In [2]:
from os.path import join
import pandas as pd

corpus_df = pd.read_csv(join(FOLDER,'corpus_df.csv'))
print(corpus_df)

      Unnamed: 0  index                                         para_array  \
0              0      0  ['Three', 'Rings', 'for', 'the', 'Elven-kings'...   
1              1      1  ['When', 'those', 'whose', 'advice', 'and', 'o...   
2              2      2  ['It', 'was', 'during', '1944', 'that,', 'leav...   
3              3      3  ['As', 'for', 'any', 'inner', 'meaning', 'or',...   
4              4      4  ['Other', 'arrangements', 'could', 'be', 'devi...   
...          ...    ...                                                ...   
4890        4890   5432  ['Elrond', 'greeted', 'them', 'gravely', 'and'...   
4891        4891   5433  ["'So", 'I', 'thought', 'too,', 'once.', 'But'...   
4892        4892   5435  ['But', 'Sam', 'was', 'now', 'sorrowful', 'at'...   
4893        4893   5436  ['But', 'to', 'Sam', 'the', 'evening', 'deepen...   
4894        4894   5437  ['He', 'drew', 'a', 'deep', 'breath.', "'Well,...   

                                               para_raw        

In [3]:
#[2]

from transformers import GPT2LMHeadModel, GPT2Tokenizer

#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#print(tokenizer)
#print(model)

In [4]:
#[3]

import torch

#Accumulated batch size
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None


In [5]:
#[5]

from torch.utils.data import Dataset, DataLoader

class Story_text(Dataset):
    def __init__(self, control_code, truncate = False, gpt2_type="gpt2", max_length=1024):
        
        self.tokenizer= GPT2Tokenizer.from_pretrained(gpt2_type)
        self.raw_para = []
        
        for row in corpus_df['para_raw']:
            self.raw_para.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|{row[:max_length]}<|endoftext|>")
            ))
            
            
        if truncate:
            self.raw_para = self.raw_para[:20000]
        self.text_count = len(self.raw_para)
        
    def __len__(self):
        return self.text_count
    
    def __getitem__(self, item):
        return self.raw_para[item]  

dataset = Story_text(corpus_df['para_raw'], truncate= True, gpt2_type= "gpt2")

In [6]:
#[6]

#dataset = Story_Text class
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    
    device = torch.device("cuda")
    model = model.cuda()
    #device = torch.device("cpu")
    #model = model.cpu()
    
    
    model.train()

    #optimizer = AdamW(model.parameters(), lr=lr)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    # replace AdamW with Adafactor
    #optimizer = Adafactor(
    #    model.parameters(),
    #    lr=1e-3,
    #    eps=(1e-30, 1e-3),
    #    clip_threshold=1.0,
    #    decay_rate=-0.8,
    #    beta1=None,
    #    weight_decay=0.0,
    #    relative_step=False,
    #    scale_parameter=False,
    #    warmup_init=False,
    #)
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            
            #print(accumulating_batch_count)
            
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [7]:
#[7]

model = train(dataset, model, tokenizer,epochs=10)

Training epoch 0
0


4895it [04:15, 19.16it/s]


Training epoch 1
tensor(1.9229, device='cuda:0', grad_fn=<NllLossBackward>)


4895it [04:10, 19.53it/s]


Training epoch 2
tensor(1.7143, device='cuda:0', grad_fn=<NllLossBackward>)


4895it [04:11, 19.46it/s]


Training epoch 3
tensor(1.8632, device='cuda:0', grad_fn=<NllLossBackward>)


4895it [04:11, 19.45it/s]


Training epoch 4
tensor(1.9527, device='cuda:0', grad_fn=<NllLossBackward>)


4895it [04:12, 19.36it/s]


In [8]:
import os
path = os.path.abspath(os.getcwd())
torch.save(model.state_dict(), join(path, FOLDER, 'trained_model.pt'))
print("saved to " + join(path, FOLDER, 'trained_model.pt'))

saved to C:\Users\tofug\Anaconda_Projects\Coen 140\final_project\Data2\trained_model.pt
